# import packages

In [109]:
import tweepy
import yaml
import json
import sqlite3
from datetime import datetime
import pandas as pd

# Read twitter Authentication Keys

In [110]:
# yaml file reader funtion
def read_yaml(file_path):
    with open(file_path, "r") as f:
        return yaml.safe_load(f)

# yaml config file path
file_path = "twitter_api_key_config.yaml"
# read from config file
api_credential = read_yaml(file_path)

# Create Twitter Authentication

In [111]:
# API authentication
auth = tweepy.OAuthHandler(api_credential["api_key"], \
                           api_credential["api_secret_token"])
auth.set_access_token(api_credential["access_token"], \
                      api_credential["access_token_secret"])
api = tweepy.API(auth, wait_on_rate_limit=True)

# Create the Database and Required Tables

In [112]:
# establish a database connection
conn = sqlite3.connect('tbt_tweets.db')
cur = conn.cursor()
create_tweet_info_table = """CREATE TABLE tweet_info(tweet_id BIGINT PRIMARY KEY, \
                                          user_id BIGINT, \
                                          tweet_lang TEXT, \
                                          tweet_time TEXT, \
                                          source TEXT, \
                                          tweet_text TEXT,\
                                          quote_count TEXT, \
                                          reply_count INT, \
                                          retweet_count INT,\
                                          tweet_favorite_count INT, \
                                          hashtags TEXT, \
                                          short_urls TEXT, \
                                          expanded_urls TEXT, \
                                          user_mentions TEXT);"""

create_user_info_table =  """CREATE TABLE user_info(user_id BIGINT PRIMARY KEY, \
                                          user_screen_name TEXT, \
                                          user_name TEXT, \
                                          user_language TEXT, \
                                          location TEXT, \
                                          profile_url TEXT, \
                                          description TEXT, \
                                          protected TEXT, \
                                          verified TEXT, \
                                          created_at TEXT, \
                                          friends_count BIGINT, \
                                          followers_count BIGINT,\
                                          favorites_count BIGINT, \
                                          statuses_count BIGINT);"""

cur.execute(create_tweet_info_table)
cur.execute(create_user_info_table)
conn.close()

OperationalError: table tweet_info already exists

# check if a table exists in the database

In [113]:
conn = sqlite3.connect('tbt_tweets.db')
cur = conn.cursor()

table_exist_query = ''' SELECT count(*) FROM sqlite_master WHERE type='table' AND name='tweet_info' '''
cur.execute(table_exist_query)
exist_result = cur.fetchone()

if exist_result[0]==1:
    print("tweet_info table exists.")
else:
    print("tweet_info table does not exist.")
conn.close()

tweet_info table exists.


# define a StreamListener Object

In [114]:
# override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, listen_time=60):
        super(MyStreamListener, self).__init__()
        self.counter = 0
        print("Initialized Tweepy StreamListener.")
        self.start_time = datetime.now()
        self.current_time = datetime.now()
        self.listen_time = listen_time
        self.unique_user_id_set = set([])
        # adding database connection code
        self.conn = sqlite3.connect('tbt_tweets.db')
        self.cur  = self.conn.cursor()
        
    def insert_data(self, data):
        tweet_object=json.loads(data) # convert "string-line" into json
        # check if json object has a key id. Otherwise continue to next.
        if 'id' in tweet_object.keys(): 
            
            # tweet object information
            tweet_id        = tweet_object['id']
            user_id         = tweet_object['user']['id']
            tweet_lang      = tweet_object['lang']
            tweet_time      = str(pd.to_datetime(tweet_object['created_at']))
            source          = tweet_object['source']
            tweet_text      = tweet_object['text']

            # tweet numeric information
            quote_count = tweet_object['quote_count']
            reply_count = tweet_object['reply_count']
            retweet_count = tweet_object['retweet_count']
            tweet_favorite_count = tweet_object['favorite_count']

            # meta-content information
            hashtags = [str(hashtag['text']) for hashtag in tweet_object['entities']['hashtags']]
            hashtags = ",".join(hashtags)
            short_urls = [str(url['url']) for url in tweet_object['entities']['urls']]
            short_urls = ",".join(short_urls)
            expanded_urls = []
            try:
                expanded_urls = [str(url['expanded_url']) for url in tweet_object['entities']['urls']]
            except:
                print('Error Message: No Expanded URL.')
            expanded_urls = ",".join(expanded_urls)

            # user interaction based informations    
            user_mentions = [str(user_mentions['id'])\
                        for user_mentions in tweet_object['entities']['user_mentions']]
            user_mentions = ",".join(user_mentions)

            tweet_info = (tweet_id, user_id, tweet_lang,\
                    tweet_time, source, tweet_text,\
                    quote_count, reply_count, retweet_count,\
                    tweet_favorite_count, hashtags, short_urls,\
                    expanded_urls, user_mentions)
            self.cur.execute("INSERT INTO tweet_info \
            VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", tweet_info)
            
            # user profile information

            if user_id in self.unique_user_id_set:
                pass
            else:
                self.unique_user_id_set.add(user_id)
                
                user_screen_name      = tweet_object['user']['screen_name']
                user_name             = tweet_object['user']['name']
                user_language         = tweet_object['user']['lang']  
                location              = tweet_object['user']['location']
                profile_url           = tweet_object['user']['url']
                description           = tweet_object['user']['description']
                protected             = tweet_object['user']['protected']
                verified              = tweet_object['user']['verified']
                created_at            = str(pd.to_datetime(tweet_object['user']['created_at']))
                friends_count         = tweet_object['user']['friends_count']
                followers_count       = tweet_object['user']['followers_count']
                favorites_count       = tweet_object['user']['favourites_count']
                statuses_count        = tweet_object['user']['statuses_count']

                user_information = (user_id, user_screen_name, user_name,\
                       user_language, location, profile_url,\
                       description, protected, verified, created_at,\
                       friends_count, followers_count,\
                       favorites_count, statuses_count)
                self.cur.execute(" INSERT INTO user_info VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", user_information)
            
            self.conn.commit()
        
    def on_data(self, data):
        self.current_time = datetime.now()
        time_elapsed = (self.current_time - self.start_time).total_seconds()
        if time_elapsed < self.listen_time:
            try:
                self.counter += 1
                """
                Changing the code here.
                Previously, we saved to Text file.
                Now, we will pass this to the Database insertor method.
                """
                # -- self.output_file.write(str(data))
                print(f"Tweet Processed: {self.counter}\n")
                self.insert_data(str(data))
                
            except Exception as e:
                print(f"On data Exception:{e}.")
        else:
            print(f"Stream listen time period ended. Total listen time: {self.listen_time} seconds.\n\n")
            print(f"Total Tweet processed: {self.counter}")
            self.conn.close()
            return False

    # handling Errors
    def on_error(self, status_code):
        print(f"status_code: {status_code}")
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

# create a stream

In [115]:
myStreamListener = MyStreamListener(listen_time=90000)
myStream = tweepy.Stream(api.auth, myStreamListener)

Initialized Tweepy StreamListener.


# start the streamer

In [120]:
keywords = ['#tbt']
try:
    print("Stream Filter")
    myStream.filter(track=keywords)
    print("DONE")
except Exception as e:
    print(f"error in stream filter {e}")

Stream Filter
Tweet Processed: 1

Tweet Processed: 2

Tweet Processed: 3

Tweet Processed: 4

Tweet Processed: 5

Tweet Processed: 6

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 7

Tweet Processed: 8

Tweet Processed: 9

Tweet Processed: 10

Tweet Processed: 11

Tweet Processed: 12

Tweet Processed: 13

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 14

Tweet Processed: 15

Tweet Processed: 16

Tweet Processed: 17

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 18

Tweet Processed: 19

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 20

Tweet Processed: 21

Tweet Processed: 22

Tweet Processed: 23

Tweet Processed: 24

Tweet Processed: 25

Tweet Processed: 26

Tweet Processed: 27

Tweet Processed: 28

Tweet Processed: 29

Tweet Processed: 30

Tweet Processed: 31

Tweet Processed: 32

Tweet Processed: 33

Tweet Processed: 34

Tweet Processed: 35

Tweet Pr

Tweet Processed: 307

Tweet Processed: 308

Tweet Processed: 309

Tweet Processed: 310

Tweet Processed: 311

Tweet Processed: 312

Tweet Processed: 313

Tweet Processed: 314

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 315

Tweet Processed: 316

Tweet Processed: 317

Tweet Processed: 318

Tweet Processed: 319

Tweet Processed: 320

Tweet Processed: 321

Tweet Processed: 322

Tweet Processed: 323

Tweet Processed: 324

Tweet Processed: 325

Tweet Processed: 326

Tweet Processed: 327

Tweet Processed: 328

Tweet Processed: 329

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 330

Tweet Processed: 331

Tweet Processed: 332

Tweet Processed: 333

Tweet Processed: 334

Tweet Processed: 335

Tweet Processed: 336

Tweet Processed: 337

Tweet Processed: 338

Tweet Processed: 339

Tweet Processed: 340

Tweet Processed: 341

Tweet Processed: 342

Tweet Processed: 343

Tweet Processed: 344

Tweet Processed: 345

Tweet Processed:

Tweet Processed: 618

Tweet Processed: 619

Tweet Processed: 620

Tweet Processed: 621

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 622

Tweet Processed: 623

Tweet Processed: 624

Tweet Processed: 625

Tweet Processed: 626

Tweet Processed: 627

Tweet Processed: 628

Tweet Processed: 629

Tweet Processed: 630

Tweet Processed: 631

Tweet Processed: 632

Tweet Processed: 633

Tweet Processed: 634

Tweet Processed: 635

Tweet Processed: 636

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 637

Tweet Processed: 638

Tweet Processed: 639

Tweet Processed: 640

Tweet Processed: 641

Tweet Processed: 642

Tweet Processed: 643

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 644

Tweet Processed: 645

Tweet Processed: 646

Tweet Processed: 647

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 648

Tweet Processed: 649

Tweet Processed: 650

Tweet Processed: 651



Tweet Processed: 925

Tweet Processed: 926

Tweet Processed: 927

Tweet Processed: 928

Tweet Processed: 929

Tweet Processed: 930

Tweet Processed: 931

Tweet Processed: 932

Tweet Processed: 933

Tweet Processed: 934

Tweet Processed: 935

Tweet Processed: 936

Tweet Processed: 937

Tweet Processed: 938

Tweet Processed: 939

Tweet Processed: 940

Tweet Processed: 941

Tweet Processed: 942

Tweet Processed: 943

Tweet Processed: 944

Tweet Processed: 945

Tweet Processed: 946

Tweet Processed: 947

Tweet Processed: 948

Tweet Processed: 949

Tweet Processed: 950

Tweet Processed: 951

Tweet Processed: 952

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 953

Tweet Processed: 954

Tweet Processed: 955

Tweet Processed: 956

Tweet Processed: 957

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 958

Tweet Processed: 959

Tweet Processed: 960

Tweet Processed: 961

Tweet Processed: 962

Tweet Processed: 963

Tweet Processed:

Tweet Processed: 1241

Tweet Processed: 1242

Tweet Processed: 1243

Tweet Processed: 1244

Tweet Processed: 1245

Tweet Processed: 1246

Tweet Processed: 1247

Tweet Processed: 1248

Tweet Processed: 1249

Tweet Processed: 1250

Tweet Processed: 1251

Tweet Processed: 1252

Tweet Processed: 1253

Tweet Processed: 1254

Tweet Processed: 1255

Tweet Processed: 1256

Tweet Processed: 1257

Tweet Processed: 1258

Tweet Processed: 1259

Tweet Processed: 1260

Tweet Processed: 1261

Tweet Processed: 1262

Tweet Processed: 1263

Tweet Processed: 1264

Tweet Processed: 1265

Tweet Processed: 1266

Tweet Processed: 1267

Tweet Processed: 1268

Tweet Processed: 1269

Tweet Processed: 1270

Tweet Processed: 1271

Tweet Processed: 1272

Tweet Processed: 1273

Tweet Processed: 1274

Tweet Processed: 1275

Tweet Processed: 1276

Tweet Processed: 1277

Tweet Processed: 1278

Tweet Processed: 1279

Tweet Processed: 1280

Tweet Processed: 1281

Tweet Processed: 1282

Tweet Processed: 1283

Tweet Proce

...

Tweet Processed: 5320

Tweet Processed: 5321

Tweet Processed: 5322

Tweet Processed: 5323

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5324

Tweet Processed: 5325

Tweet Processed: 5326

Tweet Processed: 5327

Tweet Processed: 5328

Tweet Processed: 5329

Tweet Processed: 5330

Tweet Processed: 5331

Tweet Processed: 5332

Tweet Processed: 5333

Tweet Processed: 5334

Tweet Processed: 5335

Tweet Processed: 5336

Tweet Processed: 5337

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5338

Tweet Processed: 5339

Tweet Processed: 5340

Tweet Processed: 5341

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5342

Tweet Processed: 5343

Tweet Processed: 5344

Tweet Processed: 5345

Tweet Processed: 5346

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5347

Tweet Processed: 5348

Tweet Processed: 5349

Tweet Processed: 5350

Tweet Processed: 5351

Tweet Proces

Tweet Processed: 5627

Tweet Processed: 5628

Tweet Processed: 5629

Tweet Processed: 5630

Tweet Processed: 5631

Tweet Processed: 5632

Tweet Processed: 5633

Tweet Processed: 5634

Tweet Processed: 5635

Tweet Processed: 5636

Tweet Processed: 5637

Tweet Processed: 5638

Tweet Processed: 5639

Tweet Processed: 5640

Tweet Processed: 5641

Tweet Processed: 5642

Tweet Processed: 5643

Tweet Processed: 5644

Tweet Processed: 5645

Tweet Processed: 5646

Tweet Processed: 5647

Tweet Processed: 5648

Tweet Processed: 5649

Tweet Processed: 5650

Tweet Processed: 5651

Tweet Processed: 5652

Tweet Processed: 5653

Tweet Processed: 5654

Tweet Processed: 5655

Tweet Processed: 5656

Tweet Processed: 5657

Tweet Processed: 5658

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5659

Tweet Processed: 5660

Tweet Processed: 5661

Tweet Processed: 5662

Tweet Processed: 5663

Tweet Processed: 5664

Tweet Processed: 5665

Tweet Processed: 5666

Tweet Processed: 

Tweet Processed: 5915

Tweet Processed: 5916

Tweet Processed: 5917

Tweet Processed: 5918

Tweet Processed: 5919

Tweet Processed: 5920

Tweet Processed: 5921

Tweet Processed: 5922

Tweet Processed: 5923

Tweet Processed: 5924

Tweet Processed: 5925

Tweet Processed: 5926

Tweet Processed: 5927

Tweet Processed: 5928

Tweet Processed: 5929

Tweet Processed: 5930

Tweet Processed: 5931

Tweet Processed: 5932

Tweet Processed: 5933

Tweet Processed: 5934

Tweet Processed: 5935

Tweet Processed: 5936

Tweet Processed: 5937

Tweet Processed: 5938

Tweet Processed: 5939

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5940

Tweet Processed: 5941

Tweet Processed: 5942

Tweet Processed: 5943

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5944

Tweet Processed: 5945

Tweet Processed: 5946

Tweet Processed: 5947

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 5948

Tweet Processed: 5949

Tweet 

Tweet Processed: 6214

Tweet Processed: 6215

Tweet Processed: 6216

Tweet Processed: 6217

Tweet Processed: 6218

Tweet Processed: 6219

Tweet Processed: 6220

Tweet Processed: 6221

Tweet Processed: 6222

Tweet Processed: 6223

Tweet Processed: 6224

Tweet Processed: 6225

Tweet Processed: 6226

Tweet Processed: 6227

Tweet Processed: 6228

Tweet Processed: 6229

Tweet Processed: 6230

Tweet Processed: 6231

Tweet Processed: 6232

Tweet Processed: 6233

Tweet Processed: 6234

Tweet Processed: 6235

Tweet Processed: 6236

Tweet Processed: 6237

Tweet Processed: 6238

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 6239

Tweet Processed: 6240

Tweet Processed: 6241

Tweet Processed: 6242

Tweet Processed: 6243

Tweet Processed: 6244

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet Processed: 6245

Tweet Processed: 6246

Tweet Processed: 6247

Tweet Processed: 6248

On data Exception:UNIQUE constraint failed: user_info.user_id.
Tweet 

KeyboardInterrupt: 

(1438344497218326532, 192832526, 'en', '2021-09-16 03:30:25+00:00', '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'Its called PRIVATE LIFE for a reason #thursdaymorning #Tbt https://t.co/17uTnZCSbY', '0', 0, 0, 0, 'thursdaymorning,Tbt', '', '', '')



(192832526, 'sucrepie_', 'Judith', None, 'Hungary', None, 'Agriculture| Plant based food $ smoothies recipes | Relationships| Public Relations | God fearing | Model | Research| Realist | I love love and love to be loved', '0', '0', '2010-09-20 08:21:45+00:00', 156, 133, 5347, 2320)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



user_info_column_names: ['user_id', 'user_screen_name', 'user_name', 'user_language', 'location', 'profile_url', 'description', 'protected', 'verified', 'created_at', 'friends_count', 'followers_count', 'favorites_count', 'statuses_count']


# fetch the data into a pandas dataframe

In [124]:
cur.execute("SELECT * FROM tweet_info")
tweet_info_all_result = cur.fetchall()
tweet_info_column_names = [description[0] for description in cur.description]
tweet_info_dataframe = pd.DataFrame(tweet_info_all_result, \
                                   columns=tweet_info_column_names)
print(f"No of tweet stored: {len(tweet_info_all_result)}, {tweet_info_dataframe.shape[0]}\n")



cur.execute("SELECT * FROM user_info")
user_info_all_result = cur.fetchall()
user_info_column_names = [description[0] for description in cur.description]
user_info_dataframe = pd.DataFrame(user_info_all_result, \
                                   columns=user_info_column_names)
print(f"No of user info stored: {len(user_info_all_result)}, {user_info_dataframe.shape[0]}\n")

No of tweet stored: 100160, 100160

No of user info stored: 89159, 89159



In [94]:
tweet_info_dataframe.head()

,tweet_id,user_id,tweet_lang,tweet_time,source,tweet_text,quote_count,reply_count,retweet_count,tweet_favorite_count,hashtags,short_urls,expanded_urls,user_mentions
0,1438344497218326532,192832526,en,2021-09-16 03:30:25+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Its called PRIVATE LIFE for a reason #thursday...,0,0,0,0,"thursdaymorning,Tbt",,,
1,1438344505267298307,37161598,en,2021-09-16 03:30:27+00:00,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Outta Dubai good morning fam?\n#GMITM,0,0,0,0,GMITM,,,
2,1438344511474786310,37161598,en,2021-09-16 03:30:29+00:00,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @Twin_Geez: @HomeboyzRadio @GMONEYizME Duba...,0,0,0,0,"TBT,GMITM",,,"412145250,37161598,22924994,22924994,37161598"
3,1438344530076618752,243906097,und,2021-09-16 03:30:33+00:00,"<a href=""http://twitter.com/download/iphone"" r...",RT @Recibo_CRF: 🎯 #Tbt\n\n#recibo #Flamengo ht...,0,0,0,0,"Tbt,recibo,Flamengo",,,3070684244
4,1438344566088810497,1849788637,und,2021-09-16 03:30:42+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @Recibo_CRF: 🎯 #Tbt\n\n#recibo #Flamengo ht...,0,0,0,0,"Tbt,recibo,Flamengo",,,3070684244


In [95]:
user_info_dataframe.head()

,user_id,user_screen_name,user_name,user_language,location,profile_url,description,protected,verified,created_at,friends_count,followers_count,favorites_count,statuses_count
0,192832526,sucrepie_,Judith,None,Hungary,None,Agriculture| Plant based food $ smoothies reci...,0,0,2010-09-20 08:21:45+00:00,156,133,5347,2320
1,37161598,HomeboyzRadio,Homeboyz Radio,None,Kenya,http://www.hbr.co.ke,The Urban Sound of Nairobi!,0,1,2009-05-02 07:22:29+00:00,2906,334684,169051,618003
2,243906097,guiicostavieira,Guilherme 8x🏆ᶜʳᶠ,None,"Campo Grande, Brasil",None,É o certo pelo certo. O errado não se encaixa,0,0,2011-01-28 02:13:54+00:00,641,952,81330,73372
3,1849788637,gabrielsancarr,Oak Flaᶜʳᶠ,None,"Iguaba Grande, Brasil",https://facebook.com/gabriel.carvalho.98622733...,(29/07/2016 Lolove ❤) (Casado 💍) (Pai Do Zyan)...,0,0,2013-09-09 23:40:49+00:00,1078,924,475409,32393
4,1151926051892805632,RichWeberr,Richard Weber,None,Rhuden Germany,None,Futebol@Flamengo ⚽️ Analises na área Esportiva...,0,0,2019-07-18 18:45:47+00:00,2356,2338,119108,25881


['user_id', 'user_screen_name', 'friends_count', 'followers_count', 'favorites_count', 'statuses_count']


,user_id,user_screen_name,friends_count,followers_count,favorites_count,statuses_count
0,192832526,sucrepie_,156,133,5347,2320
1,37161598,HomeboyzRadio,2906,334684,169051,618003
2,243906097,guiicostavieira,641,952,81330,73372
3,1849788637,gabrielsancarr,1078,924,475409,32393
4,1151926051892805632,RichWeberr,2356,2338,119108,25881


Returned entries: 5636
# rows user_info_filtered_dataframe: 5636


,user_id,followers_count
0,192832526,133
1,37161598,334684
2,243906097,952
3,1849788637,924
4,1151926051892805632,2338


# close the database connection

In [18]:
conn.close()